# Pleiotropy traits: rare-variant association analysis

## Aim

Run the regenie_burden on different traits for the pleiotropy project

In [1]:
# Common variables Columbia's cluster
UKBB_PATH=$HOME/UKBiobank
UKBB_yale=$HOME/UKBiobank_Yale_transfer
USER_PATH=$HOME/project
container_lmm=$HOME/containers/lmm.sif
container_marp=$HOME/containers/marp.sif
container_annovar=$HOME/containers/gatk4-annovar.sif
hearing_pheno_path=$UKBB_PATH/phenotype_files/hearing_impairment
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
formatFile_fastgwa=$USER_PATH/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=$USER_PATH/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=$USER_PATH/UKBB_GWAS_dev/data/saige_template.yml
formatFile_regenie=$USER_PATH/UKBB_GWAS_dev/data/regenie_template.yml
##bfile=$UKBB_yale/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
##unrelated_samples=$UKBB_yale/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620

# Cleaned Imputed data BGEN input
##genoFile=`echo $UKBB_yale/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
##sampleFile=$UKBB_yale/ukb39554_imputeddataset/ukb32285_imputedindiv.sample

# Non-QC'ed Exome data PLINK input (as downloaded from the UKBB)
##genoFile=`echo $UKBB_yale/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c{1..22}_b0_v1.bed`

/home/dmc2245/miniconda3/lib/python3.8/site-packages/sos_notebook/kernel.py:1334: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  msg['msg_id'] = self._parent_header['header']['msg_id']


In [2]:
# Workflows
lmm_sos=$USER_PATH/bioworkflows/GWAS/LMM.ipynb
anno_sos=$USER_PATH/bioworkflows/variant-annotation/annovar.ipynb
clumping_sos=$USER_PATH/bioworkflows/GWAS/LD_Clumping.ipynb
extract_sos=$USER_PATH/bioworkflows/GWAS/Region_Extraction.ipynb
snptogene_sos=$USER_PATH/UKBB_GWAS_dev/workflow/snptogene.ipynb

# LMM directories for imputed data
lmm_imp_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_imputed_data
lmm_imp_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_imputed_data
lmm_imp_dir_saige=$UKBB_PATH/results/SAIGE_results/results_imputed_data
lmm_imp_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_imputed_data

# LMM directories for exome data
lmm_exome_dir_fastgwa=$UKBB_PATH/results/FastGWA_results/results_exome_data
lmm_exome_dir_bolt=$UKBB_PATH/results/BOLTLMM_results/results_exome_data
lmm_exome_dir_saige=$UKBB_PATH/results/SAIGE_results/results_exome_data
lmm_exome_dir_regenie=$UKBB_PATH/results/REGENIE_results/results_exome_data

In [4]:
## LMM variables 
## Specific to Bolt_LMM
LDscoresFile=$UKBB_PATH/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=$UKBB_PATH/genetic_map_hg19_withX.txt.gz
covarMaxLevels=10
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
ylim=0

### Specific to FastGWA (depeding if you run from Yale or Columbia)
####Yale's cluster
grmFile=$UKBB_PATH/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp
####Columbia's cluster
grmFile=$UKBB_yale/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp

### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID

### Specific to REGENIE
bsize=1000
lowmem=$HOME/scratch60/
lowmem_dir=$HOME/scratch60/predictions
trait=bt
minMAC=4
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
reverse_log_p=True

## Create the anno_file, set_list_file,aaf_file and mask_files necessary for burden test

In [8]:
burden_dir=~/pleiotropy/102121_burden_files
anno_sbatch_burden=~/pleiotropy/102121_burden_files/ukb23155_200Kexomes_burdenfiles_$(date +"%Y-%m-%d").sbatch
annotated_file_hg38=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz
bim_name=~/UKBiobank/results/ukb23155_200Kexomes_annovar/091321_exome_bim_merge/ukb23155_chr1_chr22_091321.bim
job_size=1
name_prefix='ukb23155_chr1_chr22_qcexomedata_102121_burden_files'
anno_sos=$HOME/project/bioworkflows/variant-annotation/annovar.ipynb
tpl_file=$HOME/project/bioworkflows/admin/csg.yml
container_annovar=$HOME/containers/gatk4-annovar.sif

anno_args="""burden_files
    --cwd $burden_dir
    --annotated_file $annotated_file_hg38
    --bim_name $bim_name
    --name_prefix $name_prefix
    --job_size $job_size
    --container_annovar $container_annovar
    --container_lmm /mnt/mfs/statgen/containers/lmm.sif
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $anno_sos \
    --to-script $anno_sbatch_burden\
    --args "$anno_args"


INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/pleiotropy/102121_burden_files/ukb23155_200Kexomes_burdenfiles_2021-10-21.sbatch
INFO: Workflow csg (ID=w7feb221e3490aa74) is executed successfully with 1 completed step.


## Regenie burden association analysis

### 05/02/22 Asthma extended white (INCORRECT bed files and PC file)

In [11]:
lmm_dir_regenie=~/pleiotropy/050222_asthma_burden_extwhite
lmm_sbatch_regenie=~/pleiotropy/050222_asthma_burden_extwhite/asthma_200k_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank_Yale_transfer/pleiotropy_R01/phenotypesforanalysis/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_withagesex_033120
covarFile=~/UKBiobank_Yale_transfer/pleiotropy_R01/phenotypesforanalysis/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_withagesex_033120
phenoCol=ASTHMA
covarCol=SEX
qCovarCol=AGE
## QC'ed exome files for 200K
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
anno_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aaf_file
keep_gene=
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=$HOME/containers/marp.sif
container_lmm=$HOME/containers/lmm.sif 
lmm_job_size=1
ylim=0
reverse_log_p=True
numThreads=2
formatFile_regenie=$HOME/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --keep_gene $keep_gene
    --aaf_bins $aaf_bins
    --aaf_file $aaf_file
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/pleiotropy/102121_asthma_burden/asthma_200k_exomes-regenie-burden_2021-10-21.sbatch
INFO: Workflow csg (ID=wbeb5eaa15673eb03) is executed successfully with 1 completed step.


### 05/12/22 Asthma extended white

In [4]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_burden_extwhite
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_burden_extwhite/asthma_white_exomes-regenie-burden_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_White_asthma_pcs
phenoCol=ASTHMA_ICD10orself_03_28_22
covarCol=sex
qCovarCol=`echo age asm_PC{1..10}`
## QC'ed exome files for 200K
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/010622_exp_white_finalqc/cache/*.bed
anno_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/pleiotropy/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aaf_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=$HOME/containers/marp.sif
container_lmm=$HOME/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=2
formatFile_regenie=$HOME/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --sampleFile $sampleFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_bins $aaf_bins
    --aaf_file $aaf_file
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_burden_extwhite/asthma_white_exomes-regenie-burden_2022-05-12.sbatch
INFO: Workflow csg (ID=w725338aa7ba1e019) is executed successfully with 1 completed step.



### 05/12/22 Asthma Asian

In [5]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_asian
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_asian/asthma_asian_exomes-regenie-burden$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_Asian_asthma_pcs
phenoCol=ASTHMA_ICD10orself_03_28_22
covarCol=sex
qCovarCol=`echo age asm_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/021722_asian_finalqc_10157samples_444076variants/cache/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_asian/asthma_asian_exomes-regenie-burden2022-05-12.sbatch
INFO: Workflow csg (ID=we1031cdb10b8ee7b) is executed successfully with 1 completed step.



### 05/12/22 Asthma African

In [6]:
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_african
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_african/ashtma_african-regenie-burden$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_African_asthma_pcs
phenoCol=ASTHMA_ICD10orself_03_28_22
covarCol=sex
qCovarCol=`echo age asm_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/021722_african_finalqc_8591samples_351690variants/cache/*.bed
anno_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.anno_file
set_list=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.set_list_file
mask_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.mask_file
aaf_file=~/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/102121_burden_files/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.aff_file
build_mask=max
aaf_bins='0.005 0.01'
tpl_file=~/project/bioworkflows/admin/csg.yml
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
container_marp=~/containers/marp.sif
container_lmm=~/containers/lmm.sif 
lmm_job_size=1
ylim=0
k=10
reverse_log_p=True
numThreads=20
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
## Trait leave empty for qt traits
trait=bt
minMAC=1
snpannofile=~/UKBiobank/results/ukb23155_200Kexomes_annovar/2021_10_12_hg38_exome/ukb23155_chr1_chr22_091321.hg38.hg38_multianno.renamedcols.csv.gz

lmm_args="""regenie_burden
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --trait $trait
    --anno_file $anno_file
    --set_list $set_list
    --mask_file $mask_file
    --aaf_file $aaf_file
    --aaf_bins $aaf_bins
    --build_mask $build_mask
    --job_size $lmm_job_size
    --ylim $ylim
    --k $k
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --minMAC $minMAC
    --snpannofile $snpannofile
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_burden_exome/051222_asthma_african/ashtma_african-regenie-burden2022-05-12.sbatch
INFO: Workflow csg (ID=w2c6ee6fc287350f4) is executed successfully with 1 completed step.



## Regenie single variant analysis for asthma with WES

In [6]:
## All filters set to 0 because the version of the bfile has already been QC'ed previously and there is not need to do it here
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
lmm_dir_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/060822_asthma_WES_200K
lmm_sbatch_regenie=~/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/060822_asthma_WES_200K/asthma_WES_200K-regenie_$(date +"%Y-%m-%d").sbatch
phenoFile=~/UKBiobank/phenotype_files/pleiotropy/UKB_exome_White_asthma_pcs
phenoCol=ASTHMA_ICD10orself_03_28_22
covarCol=sex
qCovarCol=`echo age asm_PC{1..10}`
genoFile=`echo ~/UKBiobank/data/exome_files/project_VCF/072721_run/plink/ukb23156_c{1..22}.merged.filtered.bed`
bfile=~/UKBiobank/genotype_files_processed/010622_exp_white_finalqc/cache/*.bed
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
bsize=1000
reverse_log_p=True
numThreads=20

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --genoFile $genoFile
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_dir $lowmem_dir
    --trait $trait 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --numThreads $numThreads
    --no-annotate
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args" 

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/UKBiobank/results_pleiotropy/REGENIE_results/results_exome_data/060822_asthma_WES_200K/asthma_WES_200K-regenie_2022-06-08.sbatch
INFO: Workflow csg (ID=wf46c31d9af94966f) is executed successfully with 1 completed step.

